In [ ]:
import functions_model_definition as md
import functions_read_data as rdat
import functions_occlusion as oc
import functions_gradcam as gc
import functions_plot_heatmap as phm
import functions_slider as sl

In [ ]:
occ_size = (14, 14, 10)
occ_stride = 6
###per patient input
volume = np.expand_dims(X_train, axis = -1)[0]

###
occlusion_size = np.array(occ_size)
normalize = True
both_directions = False
invert_hm = "pred_class"
occlusion_stride = occ_stride
input_shape = (128,128,28,1)

In [ ]:
volume = volume.reshape(input_shape)

## CI_B Ontram

In [ ]:
heatmap_prob_sum = np.zeros((volume.shape[0], volume.shape[1], volume.shape[2]), np.float32)
heatmap_occ_n = np.zeros((volume.shape[0], volume.shape[1], volume.shape[2]), np.float32)

## Generate all possible occlusions
X_occ = []
xyz = []
for n, (x, y, z, vol_float) in enumerate(oc.iter_occlusion(
            volume, size = occlusion_size, stride = occlusion_stride)):
            X_occ.append(vol_float.reshape(volume.shape[0], volume.shape[1], volume.shape[2], 1)) #3 changed from 1 manually
            xyz.append((x,y,z))



X_occ_2 = np.array(X_occ)
data_loader_1 = X_occ_2
data_occ = tf.data.Dataset.from_tensor_slices((data_loader_1))
data_occ_labels = tf.data.Dataset.from_tensor_slices(np.repeat((Y_test_MRS[0],), 
                                                               repeats=len(X_occ_2), axis=0))
occ_loader = tf.data.Dataset.zip((data_occ, data_occ_labels))
data_occ_full = occ_loader.batch(5, drop_remainder = True)

In [ ]:
preds_occ = predict_ontram(mod_ontram_lin2, data = data_occ_full)

In [ ]:
for i in range(len(X_occ)):
    x,y,z = xyz[i]
    heatmap_prob_sum[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += preds_occ['pdf'][i][0]
    heatmap_occ_n[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += 1

hm_maurice = heatmap_prob_sum / heatmap_occ_n # calculate average probability per voxel'

In [ ]:
phm.plot_heatmap(np.squeeze(X_test[6], axis=-1), hm_maurice,
                version = "overlay",
                mode = "def",
                slices = (88,72,12),
                hm_colormap="jet",
                hm_positive=True,
                colorbar=True)

## CI_B & LS_X Ontram

In [ ]:
preds_occ = predict_ontram(mod_ontram_lin2, data = data_occ_full)

In [ ]:
for i in range(len(X_occ)):
    x,y,z = xyz[i]
    heatmap_prob_sum[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += preds_occ['pdf'][i][0]
    heatmap_occ_n[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += 1

hm_maurice = heatmap_prob_sum / heatmap_occ_n # calculate average probability per voxel'

In [ ]:
phm.plot_heatmap(np.squeeze(X_test[6], axis=-1), hm_maurice,
                version = "overlay",
                mode = "def",
                slices = (88,72,12),
                hm_colormap="jet",
                hm_positive=True,
                colorbar=True)

## Jonas 

In [ ]:
heatmap_prob_sum = np.zeros((volume.shape[0], volume.shape[1], volume.shape[2]), np.float32)
heatmap_occ_n = np.zeros((volume.shape[0], volume.shape[1], volume.shape[2]), np.float32)

In [ ]:
## Generate all possible occlusions
X_occ = []
xyz = []
for n, (x, y, z, vol_float) in enumerate(oc.iter_occlusion(
            volume, size = occlusion_size, stride = occlusion_stride)):
            X_occ.append(vol_float.reshape(volume.shape[0], volume.shape[1], volume.shape[2], 1)) #3 changed from 1 manually
            xyz.append((x,y,z))

In [ ]:
preds_occ_jonas = jon_softmax.predict(data_occ_full)

In [ ]:
for i in range(len(X_occ)):
    x,y,z = xyz[i]
    heatmap_prob_sum[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += preds_occ_jonas[i][0]
    heatmap_occ_n[x:x + occlusion_size[0], y:y + occlusion_size[1], z:z + occlusion_size[2]] += 1

hm_jonas = heatmap_prob_sum / heatmap_occ_n # calculate average probability per voxel'

In [ ]:
phm.plot_heatmap(np.squeeze(X_test[10], axis=-1), hm_jonas,
                version = "overlay",
                mode = "def",
                slices = (88,72,12),
                hm_colormap="jet",
                hm_positive=True,
                colorbar=True)